# Exploratory Data Analysis
-------

## Import all necessary packages

In [4]:
import numpy as np
import pandas as pd
import os
from glob import glob
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

## Examine the Dataset
Below is an example of what typical rows in the dataset look like along with a detailed explanation of each column.
Note that the filepath below are relative to the Udacity Workspace and you will need to change them to your
local filepath if you want to run this locally. [Here](https://www.kaggle.com/nih-chest-xrays/data) is the where the dataset is located.

In [5]:
# Read in the data sets
all_xray_df = pd.read_csv('/data/Data_Entry_2017.csv')
all_xray_df.sample(3)
data_sample = pd.read_csv('sample_labels.csv')
data_sample.sample(3)

#### Explanation of Columns

#### Demographic information

#### Examination of X-Ray Views (What types, what they look like, change this to text explanation)

#### Examination of Diseases (pneumonia cases, non-pnuemonia cases, number of disease per patient, intensity histograms (only on sample), change this to text explanation)

#### Co-occurrence with Pneumonia

## Important Findings
-----
